In [ ]:
%pip install textblob
%pip install tweepy
%pip install plotly==5.6.0

In [ ]:
import tweepy
import pandas as pd
import re
import plotly.express as px
import nltk
from textblob import TextBlob
from wordcloud import WordCloud, STOPWORDS
from langdetect import detect
from nltk.stem import SnowballStemmer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer
nltk.downloader.download('vader_lexicon')

In [ ]:
# Authentication
consumerKey = "x"
consumerSecret = "x"
accessToken = "x"
accessTokenSecret = "x"
auth = tweepy.OAuthHandler(consumerKey, consumerSecret)
auth.set_access_token(accessToken, accessTokenSecret)
api = tweepy.API(auth)

In [ ]:
# Polarity Class
class Polarity:
    def __init__(self):
        self.__positive = 0.0
        self.__negative = 0.0
        self.__neutral = 0.0
    
    @property
    def negative(self):
        return self.__negative
    
    @negative.setter
    def negative(self, value):
        if not isinstance(value, float):
            raise TypeError("negative must be set to an float")
        self.__negative = value
        
    @property
    def positive(self):
        return self.__positive
    
    @positive.setter
    def positive(self, value):
        if not isinstance(value, float):
            raise TypeError("positive must be set to an float")
        self.__positive = value
    
    @property
    def neutral(self):
        return self.__neutral
    
    @neutral.setter
    def neutral(self, value):
        if not isinstance(value, float):
            raise TypeError("neutral must be set to an float")
        self.__neutral = value
    
    @property
    def polarity(self):
        return self.__polarity
    
    @polarity.setter
    def polarity(self, value):
        if not isinstance(value, float):
            raise TypeError("polarity must be set to an float")
        self.__polarity = value
    
    def _percentage(self, num, denom):
        return 100 * num/denom
    
    def classify_sentiment(self, score, df_tweets, index):
        neg = score['neg']
        neu = score['neu']
        pos = score['pos']
        comp = score['compound']
        df_tweets.loc[index, 'compound'] = float(format(comp, '.1f'))
        
        if neg > pos:
            df_tweets.loc[index, 'sentiment'] = "neg"
            df_tweets.loc[index, 'neg'] = float(format(neg, '.1f'))
            self.negative += 1.0
            
        elif pos > neg:
            df_tweets.loc[index, 'sentiment'] = "pos"
            df_tweets.loc[index, 'pos'] = float(format(pos, '.1f'))
            self.positive += 1.0
            
        elif pos == neg:
            df_tweets.loc[index, 'sentiment'] = "neu"
            df_tweets.loc[index, 'neu'] = float(format(neu, '.1f'))
            self.neutral += 1.0
    
    def calculate_sentiment_percentage(self, df_tweets, index, max_tweets):
        data = [{'total': self.negative, 'percentage': self._percentage(self.negative, max_tweets)}, 
                {'total': self.positive, 'percentage': self._percentage(self.positive, max_tweets)}, 
                {'total': self.neutral, 'percentage': self._percentage(self.neutral, max_tweets)}
               ]
        return pd.DataFrame(data, index =['neg', 'pos', 'neu'], columns =['total', 'percentage'])
    

In [ ]:
# Utility Methods
def create_pi_chart(pie_data_frame, query, labels):
    title = 'Sentiment Analysis for #' + query
    fig = px.pie(pie_data_frame, values="percentage", names=labels, title=title)
    fig.show()

def clean_tweet_text(df_tweets):
    strip_symbols = lambda temp: re.sub("(@[A-Za-z0-9_]+)|#[A-Za-z0-9_]+|([^0-9A-Za-z \t])|[()!?]|\[.*?\]"," ",temp)
    strip_links = lambda temp: re.sub(r'http\S+', '', temp)
    df_tweets["text"].map(strip_symbols).map(strip_links)
    return df_tweets["text"].map(strip_symbols).map(strip_links)
    

In [ ]:

def main():
    tweet_list = []
    sid = SentimentIntensityAnalyzer()
    polar = Polarity()
    
    query = input("Enter topic or hashtag: ")
    max_tweets = int(input ("Enter numer of tweets to consider: "))
    
    for tweet in tweepy.Cursor(api.search_tweets, q=query).items(max_tweets):
        tweet_list.append(tweet.text)
        df_tweets = pd.DataFrame(tweet_list)
        df_tweets["text"] = df_tweets[0]
        clean_tweet_text(df_tweets)

    for index, row in df_tweets['text'].iteritems():    
        score = sid.polarity_scores(row)
        polar.classify_sentiment(score, df_tweets, index)
    
    sentiment_percentages = polar.calculate_sentiment_percentage(df_tweets, index, max_tweets)
    create_pi_chart(sentiment_percentages, query, sentiment_percentages.index)
    
    
if __name__ == "__main__":
    main()